## Model 2b: RF + ifscore

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../models/rf_df_test_ifscore_pred.csv',sep=';', index_col=0)

In [3]:
df.head()

,ind_var30,var36,var15,num_var4,num_var8_0,ind_var13_0,ind_var12_0,saldo_var30,ind_var39_0,num_var30_0,...,delta_imp_reemb_var17_1y3,saldo_medio_var33_hace3,num_op_var40_comer_ult3,num_var29_0,num_aport_var33_ult1,TARGET,ID,iforest_score,prob_0,prob_1
50595,1.0,1.0,45.0,2.0,0.0,0.0,0.0,781.47,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0,101019,0.304430,0.977889,0.022111
5252,1.0,3.0,60.0,2.0,0.0,1.0,0.0,240003.00,1.0,6.0,...,0.0,0.0,0.0,0.0,0.0,0,10518,0.314397,0.985631,0.014369
21956,1.0,3.0,27.0,1.0,0.0,0.0,0.0,60.00,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0,43966,0.297477,0.989750,0.010250
42714,1.0,1.0,38.0,3.0,0.0,0.0,0.0,293.70,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0,85554,0.305171,0.892683,0.107317
18494,1.0,3.0,23.0,1.0,0.0,0.0,0.0,3.00,1.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0,37070,0.297388,0.999717,0.000283


prob_0	= probabilidade de estar satisfeito  
prob_1 = probabilidade de churn

In [4]:
def ks(data=None,target=None, prob=None):
    ## finding at: https://www.listendata.com/2019/07/KS-Statistics-Python.html
    data['target0'] = 1 - data[target]
    data['bucket'] = pd.qcut(data[prob], 5)
    grouped = data.groupby('bucket', as_index = False)
    kstable = pd.DataFrame()
    kstable['min_prob'] = grouped.min()[prob]
    kstable['max_prob'] = grouped.max()[prob]
    kstable['events']   = grouped.sum()[target]
    kstable['nonevents'] = grouped.sum()['target0']
    kstable = kstable.sort_values(by="min_prob", ascending=False).reset_index(drop = True)
    kstable['event_rate'] = (kstable.events / data[target].sum()).apply('{0:.2}'.format).astype(float) * 100
    kstable['nonevent_rate'] = (kstable.nonevents / data['target0'].sum()).apply('{0:.2}'.format).astype(float) * 100
    kstable['cum_eventrate']=(kstable.events / data[target].sum()).cumsum()
    kstable['cum_noneventrate']=(kstable.nonevents / data['target0'].sum()).cumsum()
    kstable['KS'] = np.round(kstable['cum_eventrate']-kstable['cum_noneventrate'], 3) * 100

    #Formating
    kstable['cum_eventrate']= kstable['cum_eventrate'].apply('{0:.2}'.format).astype(float) * 100
    kstable['cum_noneventrate']= kstable['cum_noneventrate'].apply('{0:.2}'.format).astype(float) * 100
    kstable.index = range(1,6)
    kstable.index.rename('Decile', inplace=True)
    pd.set_option('display.max_columns', 9)
    # print(kstable)
    
    #Display KS
    from colorama import Fore
    print(Fore.RED + "KS is " + str(max(kstable['KS']))+"%"+ " at decile " + str((kstable.index[kstable['KS']==max(kstable['KS'])][0])))
    return(kstable)

In [5]:
from scipy.stats import ks_2samp
yhat = df['prob_1']
y = df['TARGET']
def ks_stat(y, yhat):
    return ks_2samp(yhat[y==1], yhat[y!=1]).statistic
ks_stat(y, yhat)

0.48299119033170035

In [6]:
results_quintil = ks(df, target='TARGET', prob='prob_1')
results_quintil

KS is 46.7% at decile 2


,min_prob,max_prob,events,nonevents,event_rate,nonevent_rate,cum_eventrate,cum_noneventrate,KS
Decile,,,,,,,,,
1,0.148437,0.914777,383,2656,63.0,18.0,63.0,18.0,44.9
2,0.045120,0.148352,132,2911,22.0,20.0,85.0,38.0,46.7
3,0.015388,0.045107,56,2984,9.2,20.0,94.0,59.0,35.5
4,0.001799,0.015383,29,3012,4.8,21.0,99.0,79.0,19.6
5,0.000000,0.001797,7,3034,1.2,21.0,100.0,100.0,-0.0


In [7]:
from sklearn.metrics import roc_auc_score

print("Roc Auc: ",roc_auc_score(df['TARGET'], df['prob_1']))
print("Gini: ",2*(roc_auc_score(df['TARGET'], df['prob_1']))-1)

Roc Auc:  0.8112140011166565
Gini:  0.6224280022333131


In [8]:
results_quintil.columns = ['min_prob','max_prob','Qtd de eventos (Target)','Qtd de não-eventos (Target)','% de eventos no Decil','% de não-eventos no Decil','% de evento acumulados','% de não-evento acumulados','KS']
results_quintil.to_csv("results_rf_if_fs1.csv",sep=';')